In [1]:
from pathlib import Path
import sys, os
from pathlib import Path
sys.path.append(os.path.abspath('./..'))

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

datadir = Path('../datadir')

df_part_now = pd.read_excel(datadir / 'participant_info.xlsx')

df_part_now.date = pd.to_datetime(df_part_now.date)
df_part_now.date = df_part_now.date.dt.strftime('%Y-%m-%d')


### Instructions

- Go to REDCAp
- Export all data as CSV
- Update path to point to file
- Run


### Sync ID columns

In [2]:
fpath = Path('/Users/psr/Downloads/OpenCapSurveyCollect_DATA_2025-07-24_1433.csv')

id_cols = ['pid', 'open_id', 'open_id_2', 'mdf_id', 'myocap_id', 'digbi_id', 'digbi_2']

df_part = df_part_now.copy()
df_redcap = pd.read_csv(fpath)
df_ids = df_redcap[['visit_date', *id_cols]]

for i, row in df_part.iterrows():
    id_match = (df_ids[id_cols] == row.pid).any(axis=1)
    date_match = (df_ids.visit_date==row.date)
    df_temp = df_ids[id_match & date_match]
    if len(df_temp) < 1:
        # print(i, 'cannot find', row.date, row.pid)
        continue
    elif len(df_temp) > 1:
        print(i, 'conflicts for', row.date, row.pid)
        print(df_temp)
        continue
    else:
        # print('success')
        df_part.at[i,'DIGBI'] = df_temp['digbi_2'].iloc[0]
    # break

df_part[df_part.DIGBI.str.len() > 0]


141 conflicts for 2023-12-06 DB-318
     visit_date  pid open_id open_id_2 mdf_id myocap_id digbi_id digbi_2
696  2023-12-06  NaN     NaN       NaN    NaN       NaN   DB-318  DB-318
697  2023-12-06  NaN     NaN       NaN    NaN       NaN      NaN  DB-318
700 conflicts for 2024-06-22 p232
     visit_date   pid open_id open_id_2 mdf_id myocap_id digbi_id digbi_2
154  2024-06-22  p232     NaN       NaN    NaN       NaN      NaN  DB-330
774  2024-06-22  p232    p323       NaN    NaN       NaN   DB-599  DB-599
701 conflicts for 2024-06-22 p235
     visit_date   pid open_id open_id_2 mdf_id myocap_id digbi_id digbi_2
113  2024-06-22  p235     NaN       NaN    NaN       NaN      NaN  DB-374
625  2024-06-22  p235     NaN       NaN    NaN       NaN      NaN  DB-232


,pid,DIGBI,date,type,location,clinician,weight,height,age,birth_year,...,time_10mwrt,time_tug_line,time_tug_cone,clock_b,time_5xsts,clock_c,time_stairs_up,unable,notes_p,todo
12,CMT001,DB-641,2024-09-06,CMT 1A,Denver CMTA,JM,72.6,1.65,NaN,1952.0,...,NaN,NaN,18.03,NaN,9.56,NaN,NaN,10mwrt,"R foot fused, not able to toe stand",NaN
13,cmt002,DB-642,2024-09-06,CMT 1A,Denver CMTA,CAS,111.1,1.78,NaN,1964.0,...,8.35,NaN,10.66,NaN,12.43,NaN,NaN,NaN,NaN,NaN
15,CMT004,DB-673,2024-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.35,NaN,6.13,NaN,9.66,NaN,NaN,NaN,NaN,NaN
16,CMT005,DB-643,2024-09-06,CMT 1A,Denver CMTA,CM,113.4,1.70,NaN,1983.0,...,6.19,NaN,7.63,NaN,8.34,NaN,NaN,NaN,unable to stand on tiptoes during toe stand,NaN
17,cmt006,DB-644,2024-09-07,unknown,Denver CMTA,NaN,54.4,1.70,NaN,2001.0,...,5.41,NaN,6.50,NaN,9.22,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,p327,DB-293,2024-06-23,DMD,Campus Gym,TD,20.4,1.17,NaN,2017.0,...,6.62,NaN,6.56,NaN,6.81,NaN,NaN,NaN,"first brooke video is the good one, 10mwrt and...",NaN
769,p328,DB-588,2024-06-23,CMT,Campus Gym,TD,25.0,1.39,NaN,2013.0,...,8.03,NaN,9.97,NaN,33.40,NaN,NaN,jump,Testing performed without braces; 5xsts took 3...,NaN
770,p329,DB-286,2024-06-23,DMD,Campus Gym,NB,71.7,1.55,NaN,2005.0,...,8.72,NaN,20.06,NaN,NaN,NaN,NaN,jump,5xsts only did one repetition (16.72s); skippe...,unable: 5xsts
771,p330,DB-540,2024-06-23,CTL,Campus Gym,PSR,45.4,1.63,NaN,2000.0,...,3.56,NaN,6.78,NaN,6.66,NaN,NaN,NaN,redid 10mwrt because slowed down the first tim...,NaN


### Sync TFT data


In [3]:
tft_col_map = {
    'time_10mwt': 'tmt_walk_time',
    'time_10mwrt': 'tmt_run_time',
    'time_5xsts': 'fsts_time',
    'time_tug_cone': 'tug_cone_time',
}

for i, row in df_part.iterrows():
    id_match = (df_redcap.digbi_2 == row.DIGBI)
    date_match = (df_redcap.visit_date == row.date)
    df_temp = df_redcap[id_match & date_match]
    
    if len(df_temp) < 1:
        # print(i, 'cannot find', row.date, row.pid)
        continue
    elif len(df_temp) > 1:
        print(i, 'conflicts for', row.date, row.pid)
        print(df_temp)
        continue
    else:
        # df_part.at[i,'DIGBI'] = df_temp['digbi_2'].iloc[0]

        for akey, bkey in tft_col_map.items():
            aval = df_part.loc[i, akey]
            bval = df_temp.iloc[0][bkey]

            if not np.isnan(df_temp.iloc[0][bkey+'_2']):
                print(row.DIGBI, row.date)
                print('Repeated trial,', bkey, '(skipping for now)')
                print()
                continue

            if np.isnan(aval) and np.isnan(bval):
                # missing TFT for this participant
                continue
            elif not np.isnan(aval) and np.isnan(bval):
                # TFT in spreadsheet not in REDCap
                print(row.DIGBI, row.date)
                print(f'REDCap missing: {akey}={aval} but {bkey}={bval}')
                print()
            elif np.isnan(aval) and not np.isnan(bval):
                # val on REDCap not in spreadsheet yet
                df_part.at[i, akey] = bval
            elif not np.isnan(aval) and not np.isnan(bval):
                if not np.isclose(aval, bval):
                    print(row.DIGBI, row.date)
                    print(f'conflict: {akey}={aval} but {bkey}={bval}')
                    print()


DB-287 2024-01-23
Repeated trial, tmt_walk_time (skipping for now)

DB-305 2023-03-15
Repeated trial, fsts_time (skipping for now)

DB-324 2024-08-10
Repeated trial, tug_cone_time (skipping for now)

DB-334 2024-08-10
Repeated trial, tug_cone_time (skipping for now)

DB-336 2024-12-04
Repeated trial, tmt_walk_time (skipping for now)

DB-336 2024-12-04
Repeated trial, tmt_run_time (skipping for now)

DB-336 2024-12-04
Repeated trial, fsts_time (skipping for now)

DB-336 2024-12-04
Repeated trial, tug_cone_time (skipping for now)

DB-350 2024-08-10
Repeated trial, fsts_time (skipping for now)

DB-356 2024-08-10
REDCap missing: time_tug_cone=12.25 but tug_cone_time=nan

DB-397 2025-02-11
Repeated trial, tmt_walk_time (skipping for now)

DB-397 2025-02-11
Repeated trial, tmt_run_time (skipping for now)

DB-397 2025-02-11
Repeated trial, fsts_time (skipping for now)

DB-397 2025-02-11
Repeated trial, tug_cone_time (skipping for now)

DB-421 2024-09-17
Repeated trial, tmt_run_time (skipping 

In [7]:
tft_col_map = {
    'time_10mwt': ['tmt_walk_time', 'tmt_walk_time_2'],
    'time_10mwrt': ['tmt_run_time', 'tmt_run_time_2', 'tmt_run_time_3'],
    'time_5xsts': ['fsts_time', 'fsts_time_2'],
    'time_tug_cone': ['tug_cone_time', 'tug_cone_time_2', 'tug_cone_time_3'],
}

df_tft_1 = df_part[['pid', 'DIGBI', 'date', *tft_col_map.keys(), 'notes_p']].copy()
df_tft_2 = df_redcap[['digbi_2', 'visit_date', *[a for b in tft_col_map.values() for a in b]]].copy()
df_tft_2 = df_tft_2.rename(columns={'digbi_2':'DIGBI', 'visit_date':'date'})

df_tft_1.dropna(subset=('DIGBI', 'date'), inplace=True)
df_tft_2.dropna(subset=('DIGBI', 'date'), inplace=True)

df_tft_combo = df_tft_2.merge(df_tft_1, on=['DIGBI', 'date'], how='outer')
df_tft_combo.to_csv('df_tft_combo.csv')


In [5]:
df_part_new = df_part.copy()
df_part_new.to_excel(datadir / 'temp/participant_info.xlsx', index=False)

df_part_new


,pid,DIGBI,date,type,location,clinician,weight,height,age,birth_year,...,time_10mwrt,time_tug_line,time_tug_cone,clock_b,time_5xsts,clock_c,time_stairs_up,unable,notes_p,todo
0,281_test,NaN,2025-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,281_test,NaN,2025-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3d17b3ee,NaN,2022-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test subject,NaN
3,58385b9a,NaN,2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No REDCap; uses arms on STS,NaN
4,6ec44398,NaN,2022-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No OpenCap trials,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804,testmdf,NaN,2022-09-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
805,testmdf,NaN,2022-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
806,testnoirin,NaN,2022-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
807,training_test,NaN,2023-11-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
